In [1]:
#Neural Part Of Speech Tagging
#We're now going to solve the same problem of POS tagging with neural networks. 

#From deep learning perspective, this is a task of predicting a sequence of outputs aligned to a sequence of inputs. 
#There are several problems that match this formulation:
#Part Of Speech Tagging - an auxuliary task for many NLP problems
#Named Entity Recognition - for chat bots and web crawlers
#Protein structure prediction - for bioinformatics

In [2]:
#%tensorflow_version 1.x

In [3]:
import nltk
import sys
import numpy as np

nltk.download('brown')
nltk.download('universal_tagset')
data = nltk.corpus.brown.tagged_sents(tagset='universal')
all_tags = ['#EOS#','#UNK#','ADV', 'NOUN', 'ADP', 'PRON', 'DET', '.', 'PRT', 'VERB', 'X', 'NUM', 'CONJ', 'ADJ']

data = np.array([ [(word.lower(),tag) for word,tag in sentence] for sentence in data ])

[nltk_data] Downloading package brown to /Users/alua/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/alua/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
<ipython-input-3-bc9847c14597>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  data = np.array([ [(word.lower(),tag) for word,tag in sentence] for sentence in data ])


In [4]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data,test_size=0.25,random_state=42)

In [5]:
from IPython.display import HTML, display
def draw(sentence):
    words,tags = zip(*sentence)
    display(HTML('<table><tr>{tags}</tr>{words}<tr></table>'.format(
                words = '<td>{}</td>'.format('</td><td>'.join(words)),
                tags = '<td>{}</td>'.format('</td><td>'.join(tags)))))
    
    
draw(data[11])
draw(data[10])
draw(data[7])

NOUN,ADP,NOUN,NOUN,NOUN,NOUN,VERB,ADV,VERB,ADP,DET,ADJ,NOUN,.,implementation,of,georgia's,automobile,title,law,was,also,recommended,by,the,outgoing,jury,.


PRON,VERB,ADP,DET,NOUN,.,VERB,NOUN,PRT,VERB,.,DET,NOUN,.,it,urged,that,the,city,``,take,steps,to,remedy,'',this,problem,.


NOUN,VERB,merger,proposed


In [6]:
#Building vocabularies

#Just like before, we have to build a mapping from tokens to integer ids. This time around, our model operates 
#on a word level, processing one word per RNN step. This means we'll have to deal with far larger vocabulary.
#Luckily for us, we only receive those words as input i.e. we don't have to predict them. This means we can have 
#a large vocabulary for free by using word embeddings.

In [7]:
from collections import Counter
word_counts = Counter()
for sentence in data:
    words,tags = zip(*sentence)
    word_counts.update(words)

all_words = ['#EOS#','#UNK#'] + list(list(zip(*word_counts.most_common(10000)))[0])

#let's measure what fraction of data words are in the dictionary
print("Coverage = %.5f" % (float(sum(word_counts[w] for w in all_words)) / sum(word_counts.values())))

Coverage = 0.92876


In [8]:
from collections import defaultdict
word_to_id = defaultdict(lambda:1, { word: i for i, word in enumerate(all_words) })
tag_to_id = { tag: i for i, tag in enumerate(all_tags)}

In [9]:
#convert words and tags into fixed-size matrix
def to_matrix(lines, token_to_id, max_len=None, pad=0, dtype='int32', time_major=False):
    """Converts a list of names into rnn-digestable matrix with paddings added after the end"""
    
    max_len = max_len or max(map(len,lines))
    matrix = np.empty([len(lines), max_len],dtype)
    matrix.fill(pad)

    for i in range(len(lines)):
        line_ix = list(map(token_to_id.__getitem__,lines[i]))[:max_len]
        matrix[i,:len(line_ix)] = line_ix

    return matrix.T if time_major else matrix


In [10]:
batch_words, batch_tags = zip(*[zip(*sentence) for sentence in data[-3:]])

print("Word ids:")
print(to_matrix(batch_words, word_to_id))
print("Tag ids:")
print(to_matrix(batch_tags, tag_to_id))

Word ids:
[[   2 3057    5    2 2238 1334 4238 2454    3    6   19   26 1070   69
     8 2088    6    3    1    3  266   65  342    2    1    3    2  315
     1    9   87  216 3322   69 1558    4    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  45   12    8  511 8419    6   60 3246   39    2    1    1    3    2
   845    1    3    1    3   10 9910    2    1 3470    9   43    1    1
     3    6    2 1046  385   73 4562    3    9    2    1    1 3250    3
    12   10    2  861 5240   12    8 8936  121    1    4]
 [  33   64   26   12  445    7 7346    9    8 3337    3    1 2811    3
     2  463  572    2    1    1 1649   12    1    4    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
Tag ids:
[[ 6  3  4  6  3  3  9  9  7 12  4  5  9  4  6  3 12  7  9  7  9  8  4  6
   3  7  6 13  3  4  6  3  9  4  3  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0

In [11]:
#Build model

#Unlike our previous lab, this time we'll focus on a high-level keras interface to recurrent neural networks. 
#It is as simple as you can get with RNN, allbeit somewhat constraining for complex tasks like seq2seq.
#By default, all keras RNNs apply to a whole sequence of inputs and produce a sequence of hidden states 
#(return_sequences=True or just the last hidden state (return_sequences=False). 
#All the recurrence is happening under the hood.
#At the top of our model we need to apply a Dense layer to each time-step independently. 
#As of now, by default keras.layers.Dense would apply once to all time-steps concatenated. 
#We use keras.layers.TimeDistributed to modify Dense layer so that it would apply across both batch and time axes.

In [12]:
import keras
import keras.layers as L

model = keras.models.Sequential()
model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.SimpleRNN(64,return_sequences=True))

#add top layer that predicts tag probabilities
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

In [13]:
#Training: in this case we don't want to prepare the whole training dataset in advance. 
#The main cause is that the length of every batch depends on the maximum sentence length within the batch. 
#This leaves us two options: use custom training code as in previous seminar or use generators.
#Keras models have a model.fit_generator method that accepts a python generator yielding one batch at a time. 
#But first we need to implement such generator:

In [14]:
from keras.utils.np_utils import to_categorical
BATCH_SIZE=32
def generate_batches(sentences,batch_size=BATCH_SIZE,max_len=None,pad=0):
    assert isinstance(sentences,np.ndarray),"Make sure sentences is q numpy array"
    
    while True:
        indices = np.random.permutation(np.arange(len(sentences)))
        for start in range(0,len(indices)-1,batch_size):
            batch_indices = indices[start:start+batch_size]
            batch_words,batch_tags = [],[]
            for sent in sentences[batch_indices]:
                words,tags = zip(*sent)
                batch_words.append(words)
                batch_tags.append(tags)

            batch_words = to_matrix(batch_words,word_to_id,max_len,pad)
            batch_tags = to_matrix(batch_tags,tag_to_id,max_len,pad)

            batch_tags_1hot = to_categorical(batch_tags,len(all_tags)).reshape(batch_tags.shape+(-1,))
            yield batch_words,batch_tags_1hot

In [15]:
#Callbacks: Another thing we need is to measure model performance. The tricky part is not to count 
#accuracy after sentence ends (on padding) and making sure we count all the validation data exactly once.
#While it isn't impossible to persuade Keras to do all of that, we may as well write our own callback that does that.
#Keras callbacks allow you to write a custom code to be ran once every epoch or every minibatch. 
#We'll define one via LambdaCallback

In [16]:
def compute_test_accuracy(model):
    test_words,test_tags = zip(*[zip(*sentence) for sentence in test_data])
    test_words,test_tags = to_matrix(test_words,word_to_id),to_matrix(test_tags,tag_to_id)

    #predict tag probabilities of shape [batch,time,n_tags]
    predicted_tag_probabilities = model.predict(test_words,verbose=1)
    predicted_tags = predicted_tag_probabilities.argmax(axis=-1)

    #compute accurary excluding padding
    numerator = np.sum(np.logical_and((predicted_tags == test_tags),(test_words != 0)))
    denominator = np.sum(test_words != 0)
    return float(numerator)/denominator


class EvaluateAccuracy(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        sys.stdout.flush()
        print("\nMeasuring validation accuracy...")
        acc = compute_test_accuracy(self.model)
        print("\nValidation accuracy: %.5f\n"%acc)
        sys.stdout.flush()

In [17]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
1343/1343 [==============================] - 15s 11ms/step - loss: 0.6012

Measuring validation accuracy...
448/448 [==============================] - 3s 6ms/step

Validation accuracy: 0.94023

Epoch 2/5
1343/1343 [==============================] - 15s 11ms/step - loss: 0.0604

Measuring validation accuracy...
448/448 [==============================] - 3s 6ms/step

Validation accuracy: 0.94456

Epoch 3/5
1343/1343 [==============================] - 15s 11ms/step - loss: 0.0510

Measuring validation accuracy...
448/448 [==============================] - 3s 6ms/step

Validation accuracy: 0.94662

Epoch 4/5
1343/1343 [==============================] - 15s 11ms/step - loss: 0.0465

Measuring validation accuracy...
448/448 [==============================] - 3s 6ms/step

Validation accuracy: 0.94567

Epoch 5/5
1343/1343 [==============================] - 15s 11ms/step - loss: 0.0421

Measuring validation accuracy...
448/448 [==============================] - 3s 6ms/step

Validation

In [18]:
acc = compute_test_accuracy(model)
print("Final accuracy: %.5f"%acc)

assert acc>0.94, "Keras has gone on a rampage again, please contact course staff."

448/448 [==============================] - 3s 6ms/step
Final accuracy: 0.94533


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          500100    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, None, 64)          7360      
_________________________________________________________________
time_distributed (TimeDistri (None, None, 14)          910       
Total params: 508,370
Trainable params: 508,370
Non-trainable params: 0
_________________________________________________________________


In [101]:
#Going bidirectional

#Since we're analyzing a full sequence, it's legal for us to look into future data.
#A simple way to achieve that is to go both directions at once, making a bidirectional RNN.
#In Keras you can achieve that both manually (using two LSTMs and Concatenate) and by using 
#keras.layers.Bidirectional.
#This one works just as TimeDistributed we saw before: you wrap it around a recurrent layer 
#(SimpleRNN now and LSTM/GRU later) and it actually creates two layers under the hood.
#Your first task is to use such a layer our POS-tagger.

In [112]:
#Define a model that utilizes bidirectional SimpleRNN
model = keras.models.Sequential()

model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.SimpleRNN(64,return_sequences=True))



In [113]:

#add bidirectional layer
#add top layer that predicts tag probabilities

model.add(L.Bidirectional(L.LSTM(10, return_sequences=True), merge_mode='concat', weights=None, backward_layer=None))
model.add(L.Dense(len(all_tags),activation='softmax'))

In [114]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, None, 50)          500100    
_________________________________________________________________
simple_rnn_16 (SimpleRNN)    (None, None, 64)          7360      
_________________________________________________________________
bidirectional_21 (Bidirectio (None, None, 20)          6000      
_________________________________________________________________
dense_13 (Dense)             (None, None, 14)          294       
Total params: 513,754
Trainable params: 513,754
Non-trainable params: 0
_________________________________________________________________


In [115]:
model.compile('adam','categorical_crossentropy')


In [116]:

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5
1343/1343 [==============================] - 39s 27ms/step - loss: 0.7689

Measuring validation accuracy...
448/448 [==============================] - 8s 18ms/step

Validation accuracy: 0.94579

Epoch 2/5
1343/1343 [==============================] - 35s 26ms/step - loss: 0.0689

Measuring validation accuracy...
448/448 [==============================] - 7s 16ms/step

Validation accuracy: 0.95453

Epoch 3/5
1343/1343 [==============================] - 35s 26ms/step - loss: 0.0475

Measuring validation accuracy...
448/448 [==============================] - 7s 17ms/step

Validation accuracy: 0.95844

Epoch 4/5
1343/1343 [==============================] - 37s 28ms/step - loss: 0.0402

Measuring validation accuracy...
448/448 [==============================] - 7s 17ms/step

Validation accuracy: 0.95778

Epoch 5/5
1343/1343 [==============================] - 36s 27ms/step - loss: 0.0359

Measuring validation accuracy...
448/448 [==============================] - 7s 17ms/step

Valid

In [118]:
acc = compute_test_accuracy(model)
print("\nFinal accuracy: %.5f"%acc)

assert acc>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

448/448 [==============================] - 7s 16ms/step

Final accuracy: 0.96011
Well done!


In [ ]:
#Task I: Structured loss functions (more bonus points)
#Since we're tagging the whole sequence at once, we might as well train our network to do so. 
#Remember linear CRF from the lecture? You can also use it as a loss function for your RNN
#There's more than one way to do so, but we'd recommend starting with Conditional Random Fields
#You can plug CRF as a loss function and still train by backprop. 
#Alternatively, you can condition your model on previous tags (make it autoregressive) and perform 
#beam search over that model.

In [172]:
model = keras.models.Sequential()

model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
base = model.add(L.SimpleRNN(64,return_sequences=True))

model.add(L.Bidirectional(L.LSTM(10, return_sequences=True), merge_mode='concat', weights=None, backward_layer=None))
base = model.add(L.Dense(len(all_tags),activation='softmax'))

In [192]:
from distutils.sysconfig import get_python_lib
print(get_python_lib())

/opt/anaconda3/lib/python3.8/site-packages


In [193]:
from crf import CRFLayer


ImportError: cannot import name 'CRFLayer' from 'crf' (/opt/anaconda3/lib/python3.8/site-packages/crf/__init__.py)

###### 